In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from time import time 

from fastai import *
from fastai.basic_data import *
from fastai.basic_data import *
from fastai.tabular import *
from fastai.basic_data import DataBunch
from fastai.tabular import TabularModel

import cudf

import nvtabular as nvt
from nvtabular.ops import Normalize, FillMissing, Categorify, get_embedding_sizes
from nvtabular.torch_dataloader import AsyncTensorBatchDatasetItr, DLDataLoader

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
%load_ext snakeviz
# load snakeviz if you want to run profiling

<h3> Dataset Gathering: Define files in the training and validation datasets. </h3>

In [ ]:
# data_path = '/rapids/notebooks/jperez/Documents/ds-itr/examples/'
data_path = '/datasets/outbrain/jp_out/output/'
#df_test = 'test/'
df_valid = 'validation_feature_vectors_integral.csv/'
df_train = 'train_feature_vectors_integral_eval.csv/'

train_set = [data_path + df_train + x for x in os.listdir(data_path + df_train) if x.startswith("part")][:20] 
valid_set = [data_path + df_valid + x for x in os.listdir(data_path + df_valid) if x.startswith("part")][:5] 

In [ ]:
len(train_set), len(valid_set)

<h4>Grab column information</h4>

In [ ]:
cols = open(data_path + 'train_feature_vectors_integral_eval.csv.header').read().splitlines()

In [ ]:
cat_names = ['display_id', 'is_leak', 'doc_event_id', 'ad_id', 'doc_id', 'doc_ad_entity_id', 'doc_event_entity_id', 'doc_event_entity_id', 'doc_ad_source_id', 'doc_event_source_id', 'event_geo_location', 'ad_advertiser', 'event_country_state', 'doc_ad_publisher_id', 'doc_event_publisher_id', 'doc_ad_topic_id', 'doc_event_topic_id', 'event_country', 'doc_ad_category_id', 'doc_event_category_id', 'event_hour', 'event_platform', 'traffic_source', 'event_weekend', 'user_has_already_viewed_doc']
cont_names =  ['pop_ad_id_conf', 'pop_document_id_conf', 'user_doc_ad_sim_categories_conf', 'user_doc_ad_sim_topics_conf', 'pop_publisher_id_conf', 'pop_advertiser_id_conf', 'pop_campaign_id_conf', 'pop_source_id_conf', 'pop_entity_id_conf', 'pop_topic_id_conf', 'pop_category_id_conf', 'pop_ad_id', 'pop_document_id', 'pop_publisher_id', 'pop_advertiser_id', 'pop_campaign_id', 'pop_source_id', 'pop_entity_id', 'pop_topic_id', 'pop_category_id', 'user_doc_ad_sim_categories', 'user_doc_ad_sim_topics', 'user_doc_ad_sim_entities', 'doc_event_doc_ad_sim_categories', 'doc_event_doc_ad_sim_topics', 'doc_event_doc_ad_sim_entities', 'user_views', 'ad_views', 'doc_views', 'doc_event_days_since_published', 'doc_event_hour', 'doc_ad_days_since_published'] #+ [i for i in ds.columns if i not in cat_names and i not in ['label']]
cat_names = [name for name in cat_names if name in cols]
cont_names = [name for name in cont_names if name in cols]


<h3>Preprocessing:</h3> <p>Select operations to perform, create the Preprocessor object, create dataset iterator object and collect the stats on the training dataset</p>

In [ ]:
%%time
proc = nvt.Workflow(cat_names=cat_names, cont_names=cont_names, label_name=['label'])

In [ ]:
proc.add_cont_preprocess([FillMissing(), Normalize()])
proc.add_cat_preprocess(Categorify())

In [ ]:
%%time
trains_itrs = nvt.Dataset(train_set,names=cols, engine='csv')
valids_itrs = nvt.Dataset(valid_set,names=cols, engine='csv')

In [ ]:
output_path_train = './outbrains/train'
output_path_valid = './outbrains/valid'

In [ ]:
%%time
proc.apply(trains_itrs, apply_offline=True, record_stats=True, output_path=output_path_train, shuffle=False)

In [ ]:
%%time
proc.apply(valids_itrs, apply_offline=True, record_stats=False, output_path=output_path_valid, shuffle=False)

In [ ]:
new_train_set = [os.path.join(output_path_train, x) for x in os.listdir(output_path_train) if x.endswith("parquet")]
new_valid_set = [os.path.join(output_path_valid, x) for x in os.listdir(output_path_valid) if x.endswith("parquet")]

<h5>Gather embeddings using statistics gathered in the Read phase.</h5>

In [ ]:
embeddings = list(get_embedding_sizes(proc).values())

In [ ]:
embeddings

<h5>Create the file iterators using the FileItrDataset Class.</h5>

In [ ]:
%%time
t_batch_sets = nvt.Dataset(new_train_set, engine='parquet') 
v_batch_sets = nvt.Dataset(new_valid_set, engine='parquet')

In [ ]:
train_dataset = AsyncTensorBatchDatasetItr(t_batch_sets, 
                                           batch_size=100000, 
                                           cats=cat_names, 
                                           conts=cont_names, 
                                           labels=["label"])
valid_dataset = AsyncTensorBatchDatasetItr(v_batch_sets, 
                                           batch_size=100000, 
                                           cats=cat_names, 
                                           conts=cont_names, 
                                           labels=["label"])

<h5>Use the Deep Learning Collator to create a collate function to pass to the dataloader.</h5>

In [ ]:
def gen_col(batch):
    batch = batch[0]
    return (batch[0], batch[1]), batch[2].long()

In [ ]:
%%time
t_data = DLDataLoader(train_dataset, collate_fn=gen_col, pin_memory=False, num_workers=0)
v_data = DLDataLoader(valid_dataset, collate_fn=gen_col, pin_memory=False, num_workers=0)

<h4>After creating the Dataloaders you can leverage fastai framework to create Machine Learning models</h4>

In [ ]:
databunch = DataBunch(t_data, v_data, collate_fn=gen_col, device="cuda")

In [ ]:
%%time
model = TabularModel(emb_szs = embeddings, n_cont=len(cont_names), out_sz=2, layers=[512,256])

learn =  Learner(databunch, model, metrics=[accuracy])
learn.loss_func = torch.nn.CrossEntropyLoss()


In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(show_moms=True, suggestion=True)

In [ ]:
learning_rate = 2.75e-2
epochs = 1

In [ ]:
start = time()
learn.fit_one_cycle(epochs,learning_rate)
t_final = time() - start 

In [ ]:
t_final